# Install 

In [ ]:
! pip install transformers datasets evaluate
! pip install sacrebleu
! pip install googletrans==3.1.0a0
! pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from datasets import load_dataset, DatasetDict, Dataset
import evaluate
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import DataCollatorForSeq2Seq
from transformers import pipeline
from googletrans import Translator

# Loading Dataset

In [ ]:
dataset = load_dataset("aslg_pc12")
dataset = dataset["train"].train_test_split(train_size=0.8)

# Back Translation

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")

source_lang = "text"
target_lang = "gloss"


def preprocess_function_backtrans(examples):
    inputs = examples[source_lang]
    targets = examples[target_lang]
    model_inputs = tokenizer(inputs,  text_target=targets, max_length=128, truncation=True)
    print(type(inputs))
    return model_inputs

tokenized_data = dataset.map(preprocess_function_backtrans, batched=True)


In [ ]:
sacrebleu = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = sacrebleu.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
training_args = Seq2SeqTrainingArguments(
    output_dir="transformer_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
trainer.push_to_hub()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("fuyulinh04/transformer_model")

model = AutoModelForSeq2SeqLM.from_pretrained("fuyulinh04/transformer_model")

In [ ]:
translator = pipeline("translation", model=model, tokenizer = tokenizer)

prediction = []
answer = []

def augmenting_data():
  for i in range(0, 5000):
    text = dataset["train"][i]["text"]
    prediction.append(translator(text))

In [7]:
augmenting_data()

# Training Model

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [17]:
def merge_dataset():
  for i in range(5000):
    dataset["train"][i]["gloss"] = prediction[i]

merge_dataset()

In [18]:
source_lang = "gloss"
target_lang = "text"


def preprocess_function(examples):
    inputs = prediction
    targets = examples[target_lang]
    model_inputs = tokenizer(inputs,  text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [ ]:
tokenized_data = dataset.map(preprocess_function, batched=True)

In [ ]:
sacrebleu = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = sacrebleu.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

In [14]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="transformer_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

In [20]:
trainer.push_to_hub()

Saving model checkpoint to transformer_model
Configuration saved in transformer_model/config.json
Model weights saved in transformer_model/pytorch_model.bin
tokenizer config file saved in transformer_model/tokenizer_config.json
Special tokens file saved in transformer_model/special_tokens_map.json
Several commits (3) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.30k/231M [00:00<?, ?B/s]

Upload file runs/Jan05_03-49-53_c33bb0827e5d/events.out.tfevents.1672890596.c33bb0827e5d.1203.2:  33%|###3    …

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/fuyulinh04/transformer_model
   0540b2d..9b45666  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/fuyulinh04/transformer_model
   0540b2d..9b45666  main -> main

To https://huggingface.co/fuyulinh04/transformer_model
   9b45666..cb8e0e2  main -> main

   9b45666..cb8e0e2  main -> main



'https://huggingface.co/fuyulinh04/transformer_model/commit/9b45666feb7c1e89860c22557fbbed6660298478'

# Load Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("junowhite/transformer_model")

model = AutoModelForSeq2SeqLM.from_pretrained("junowhite/transformer_model")

In [ ]:
translators = Translator()

In [ ]:
translator = pipeline("translation", model=model, tokenizer = tokenizer)

prediction = []
answer = []

def testing_data():
  for i in range(7):
    text = dataset["train"][i]["gloss"]
    answer.append(dataset["train"][i]["text"])
    prediction.append(translators.translate(translator(text)[0]["translation_text"]).text)

In [ ]:
testing_data()
print(prediction)
print(answer)